In [0]:
import logging
import os
import pickle

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [0]:
URL = "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv"

In [37]:
subscribers = pd.read_csv(URL)
subscribers = subscribers.drop(['y_no'],axis=1)
subscribers.head(10)

,Unnamed: 0,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_yes
0,0,56,1,999,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
1,1,57,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
2,2,37,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
3,3,40,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
4,4,56,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
5,5,45,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
6,6,59,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
7,7,41,1,999,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
8,8,24,1,999,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
9,9,25,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0


In [17]:
subscribers["y_yes"].value_counts()

0    36548
1     4640
Name: y_yes, dtype: int64

In [20]:
train_data, test_data = np.split(subscribers.sample(frac=1, random_state=1729), [int(0.7 * len(subscribers))])
print(train_data.shape, test_data.shape)

target = train_data['y_yes']
train = train_data.drop(['y_yes'],axis=1)
test = test_data.drop(['y_yes'],axis=1)

xgtrain = xgb.DMatrix(train.values, target.values)
xgtest = xgb.DMatrix(test.values)

(28831, 61) (12357, 61)


In [22]:
    print("\tTrain model")
    param = {
      'max_depth':5,
      'eta':0.2,
      'gamma':4,
      'min_child_weight':6,
      'subsample':0.8,
      'silent':0,
      'objective':'binary:logistic'
    }
    num_round=100
    bst = xgb.train(param, xgtrain, num_round)

	Train model


In [33]:
preds = bst.predict(xgtest)
print(type(preds), len(preds), preds.shape, preds[0:10])

<class 'numpy.ndarray'> 12357 (12357,) [0.05135508 0.06220138 0.03742848 0.28653497 0.1318373  0.0534056
 0.03189127 0.03386365 0.03137312 0.03481041]


In [0]:
#Evaluate Test Set: how to include metrics, how to do preprocessing step, what's the temp bucket?

In [36]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(preds), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.8%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10718)    37% (218)
Purchase        9% (1044)     63% (377) 



In [0]:
y_target = test_data['y_yes']
y_preds = np.round(preds).astype(int)

In [56]:
f1 = metrics.f1_score(y_target, y_preds)
precision = metrics.precision_score(y_target, y_preds)
recall = metrics.recall_score(y_target, y_preds)

print(f1, precision, recall)

0.3740079365079365 0.6336134453781512 0.2653061224489796


In [40]:
    #bedrock = BedrockApi(logging.getLogger(__name__))
    #bedrock.log_metric("Accuracy", p)
    #bedrock.log_metric("Precision", precision)
    #bedrock.log_metric("Recall", recall)
    #bedrock.log_metric("F1 score", f1)

218
